In [1]:
import turicreate

In [2]:
song_data = turicreate.SFrame('song_data.sframe/')

In [36]:
song_data.head(3)
song_data.shape

(1116609, 6)

## 1. Counting unique users for singers

In [10]:
KW = song_data[song_data['artist'] == 'Kanye West']
len(kw['user_id'].unique())

2522

In [11]:
FF = song_data[song_data['artist'] == 'Foo Fighters']
len(FF['user_id'].unique())

2055

In [9]:
TS = song_data[song_data['artist'] == 'Taylor Swift']
len(TS['user_id'].unique())

3246

In [15]:
LG = song_data[song_data['artist'] == 'Lady GaGa']
len(LG['user_id'].unique())

2928

## 2. Find the most popular and least popular artist

In [30]:
count = song_data.groupby('artist', operations = {'total_count' : 
                                                         turicreate.aggregate.SUM('listen_count')})

In [31]:
count.sort('total_count', ascending = False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [32]:
count.sort('total_count', ascending = True)

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Boggle Karaoke,30
Diplo,30
harvey summers,31
Nâdiya,36
Aneta Langerova,38
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Jody Bernal,38


## 3. Train an item-similarity recommender

In [33]:
train_data, test_data = song_data.random_split(0.8, seed=0)

In [34]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 397.202ms                           | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.42996s

### Make predictions on first 10000 users in test set

In [37]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [39]:
subset_test_users_recommend = personalized_model.recommend(users = subset_test_users, k=1)

recommendations finished on 1000/10000 queries. users per second: 31859.3

recommendations finished on 2000/10000 queries. users per second: 41840.1

recommendations finished on 3000/10000 queries. users per second: 48665

recommendations finished on 4000/10000 queries. users per second: 53618.6

recommendations finished on 5000/10000 queries. users per second: 56670.1

recommendations finished on 6000/10000 queries. users per second: 58375.4

recommendations finished on 7000/10000 queries. users per second: 60080.7

recommendations finished on 8000/10000 queries. users per second: 59778.2

recommendations finished on 9000/10000 queries. users per second: 60712.4

recommendations finished on 10000/10000 queries. users per second: 60132.3

### Find the most recommended song and least recommeded song to 1000 users

In [41]:
count_song = subset_test_users_recommend.groupby('song', operations = {'count' : 
                                                         turicreate .aggregate.COUNT()})

song,count
Hoodoo - Muse,4
Nomenclature - AndrewBird ...,1
Billionaire [feat. BrunoMars] (Explicit Album ...,2


In [42]:
count_song.sort('count', ascending = False)

song,count
Undo - Björk,433
Secrets - OneRepublic,357
Revelry - Kings Of Leon,231
You're The One - DwightYoakam ...,158
Fireflies - CharttraxxKaraoke ...,110
Hey_ Soul Sister - Train,103
Sehr kosmisch - Harmonia,84
Horn Concerto No. 4 in Eflat K495: II. Romance ...,79
OMG - Usher featuringwill.i.am ...,77
The Scientist - Coldplay,52


In [43]:
count_song.sort('count', ascending = True)

song,count
Yes - Beyoncé,1
Nomenclature - AndrewBird ...,1
Evil Deeds - Eminem,1
Wish You Were Here -Incubus ...,1
Superstition - The Kills,1
Quutamo - Apocalyptica,1
Product Man - Swizz Beatz,1
Cry Me A River - JustinTimberlake ...,1
Harder Better FasterStronger - Daft Punk ...,1
The Diary - HollywoodUndead ...,1
